In [1]:
import { SystemMessagePromptTemplate, HumanMessagePromptTemplate ,ChatPromptTemplate} from "@langchain/core/prompts";
let translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(`你是一个专业的翻译员，你的任务是从{source_lang}翻译成{target_lang}。`);
let userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate('请翻译这句话{text}')

let chatPrompt = await ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
])
let fromattedChatPrompt = await chatPrompt.formatMessages({
  source_lang: '中文',
  target_lang: '韩语',
  text: '你好，世界'
})
console.log(fromattedChatPrompt)

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你是一个专业的翻译员，你的任务是从中文翻译成韩语。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你是一个专业的翻译员，你的任务是从中文翻译成韩语。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "请翻译这句话你好，世界",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "请翻译这句话你好，世界",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [2]:
import { load } from "dotenv";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";

const env = await load();
const process = { env };

const chatModel = new ChatOpenAI({
    configuration: {
        baseURL: process.env.baseURL,
    },
});
const outputPraser = new StringOutputParser();

const chain = chatPrompt.pipe(chatModel).pipe(outputPraser);
await chain.invoke({
    source_lang: "中文",
    target_lang: "韩文",
    text: "你好，世界",
});


"안녕하세요, 세상"

组合多个Prompt

In [3]:
import { PromptTemplate, PipelinePromptTemplate } from "@langchain/core/prompts";
let getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
};
let fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是{date}，你的主人的信息是{info}，
根据上下文，完成主人的需求
{task}
`);
const datePrompt = PromptTemplate.fromTemplate(`{date},现在是{period}`);
let periodPrompt = await datePrompt.partial({
    date: getCurrentDateStr,
});

let infoPrompt = PromptTemplate.fromTemplate(`姓名是{name}，性别是{gender}`);

const taskPrompt = PromptTemplate.fromTemplate(`我想吃{period}的{food}。
  再重复一遍我的信息{info}`);

let composedPrompt = new PipelinePromptTemplate({
    pipelinePrompts: [
        {
            name: "date",
            prompt: periodPrompt,
        },
        {
            name: "info",
            prompt: infoPrompt,
        },
        {
            name: "task",
            prompt: taskPrompt,
        },
    ],
    finalPrompt: fullPrompt,
});

let formattedPrompt = await composedPrompt.format({
    period: "早上",
    name: "张三",
    gender: "male",
    food: "lemon",
});
console.log(formattedPrompt);



你是一个智能管家，今天是2024/6/13,现在是早上，你的主人的信息是姓名是张三，性别是male，
根据上下文，完成主人的需求
我想吃早上的lemon。
  再重复一遍我的信息姓名是张三，性别是male

